In [1]:
# Compare change in median/mean income, i.e. increase, decrease by groups i.e. men 65+ from 2015 to 2022

# Compare change in specific pesticide sales as above

#

In [17]:
import glob
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot
from pathlib import Path

from scipy.stats import kstest
from scipy.stats import shapiro
from scipy.stats import lognorm
pd.options.display.max_rows = 300
pd.options.display.max_columns = 50

In [3]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns


In [18]:
rds = pd.read_csv(r"C:\Users\cianw\Documents\dataAnalytics\CA2\Data\Datasets\referenceDataSet.csv")
rds.head(10)

,Unnamed: 0,subst_cat,geo,TIME_PERIOD,harmRiskInd,harmRiskInd_next,increase,forAnalysis,nace_r2,numOrganicProcessors,areaUsedForOrganic_PCT,gini,birdBiodiversityIndex,emplyomentRate_F,emplyomentRate_M,emplyomentRate_T,MED_EF,MEI_EF,MED_EM,MEI_EM,MED_ET,MEI_ET,N_use_tonne,P_use_tonne,productivity,productivity_unit,waste,waste_unit,mostFrequentPest,pest_KG
0,0,HRI1,AT,2011,102,106.0,1,1,NaN,NaN,19.60,27.4,67.4,69.2,79.2,74.2,20949.0,23444.0,21884.0,24422.0,21463.0,23922.0,98161.0,9947.0,1.6311,Purchase Power Standard Per KG,5647.5,KG per Capita,F,1544218.0
1,1,HRI1,AT,2012,106,92.0,0,1,NaN,NaN,18.62,27.6,68.4,69.6,79.3,74.4,21183.0,23762.0,22519.0,25112.0,21807.0,24423.0,107895.0,12301.0,1.7368,Purchase Power Standard Per KG,5699.0,KG per Capita,F,1634371.0
2,2,HRI1,AT,2013,92,95.0,1,1,NaN,NaN,18.40,27.0,62.8,70.0,79.1,74.6,21629.0,23757.0,22750.0,25000.0,22073.0,24366.0,110626.0,14234.0,1.7932,Purchase Power Standard Per KG,6118.0,KG per Capita,F,1492799.0
3,3,HRI1,AT,2014,95,101.0,1,1,NaN,NaN,19.35,27.6,59.7,70.1,78.3,74.2,22712.0,25535.0,23741.0,26649.0,23211.0,26080.0,121562.0,14110.0,1.8260,Purchase Power Standard Per KG,6537.0,KG per Capita,F,1641055.0
4,4,HRI1,AT,2015,101,115.0,1,1,NaN,NaN,20.30,27.2,62.8,70.2,78.4,74.3,22741.0,25228.0,23856.0,26715.0,23260.0,25958.0,124078.0,12495.0,1.9358,Purchase Power Standard Per KG,6772.5,KG per Capita,F,2130973.0
5,5,HRI1,AT,2016,115,116.0,1,1,NaN,NaN,21.25,27.2,58.3,70.9,78.7,74.8,23202.0,25452.0,24111.0,26676.0,23694.0,26054.0,132031.0,15451.0,1.8986,Purchase Power Standard Per KG,7008.0,KG per Capita,F,2006542.0
6,6,HRI1,AT,2017,116,129.0,1,1,NaN,NaN,23.37,27.9,60.7,71.4,79.4,75.4,24433.0,27134.0,25225.0,28139.0,24752.0,27629.0,111884.0,11789.0,1.9554,Purchase Power Standard Per KG,7218.0,KG per Capita,F,1991638.0
7,7,HRI1,AT,2018,129,121.0,0,1,NaN,NaN,24.08,26.8,55.3,71.7,80.7,76.2,24675.0,27245.0,25922.0,28381.0,25176.0,27804.0,100096.0,13110.0,2.0265,Purchase Power Standard Per KG,7428.0,KG per Capita,F,2268659.0
8,8,HRI1,AT,2019,121,135.0,1,1,NaN,NaN,25.33,27.5,61.6,72.4,81.2,76.8,25241.0,28069.0,26155.0,29082.0,25729.0,28568.0,102812.0,13238.0,2.0765,Purchase Power Standard Per KG,7578.0,KG per Capita,F,2068059.0
9,9,HRI1,AT,2020,135,NaN,0,0,NaN,NaN,25.69,27.0,62.9,70.6,79.0,74.8,26348.0,29056.0,26958.0,29963.0,26555.0,29503.0,117321.0,12208.0,1.9472,Purchase Power Standard Per KG,7728.0,KG per Capita,I,2362720.0


In [45]:
harmRiskInd = rds[rds.columns[rds.columns.isin(['TIME_PERIOD', 'geo', 'harmRiskInd'])]]

hri_Other = harmRiskInd[harmRiskInd['geo'] != 'IE']
hri_Other_summary = hri_Other.groupby(['TIME_PERIOD'])['harmRiskInd'].agg(['mean', 'median', 'count', 'std'])

hri_Ire = harmRiskInd[harmRiskInd['geo'] == 'IE']
print(hri_Other_summary)
print(hri_Ire)

                   mean  median  count        std
TIME_PERIOD                                      
2011         108.148148   102.0     27  28.568008
2012          99.703704   101.0     27  15.060940
2013          92.148148    94.0     27  17.087341
2014          90.074074    96.0     27  24.902774
2015          93.296296   101.0     27  25.154622
2016          93.777778    92.0     27  29.020329
2017          87.185185    86.0     27  26.343934
2018          85.037037    82.0     27  25.782346
2019          81.185185    78.0     27  26.740976
2020          83.500000    76.5     26  32.075224
    geo  TIME_PERIOD  harmRiskInd
140  IE         2011          120
141  IE         2012           84
142  IE         2013           96
143  IE         2014          109
144  IE         2015          114
145  IE         2016          105
146  IE         2017           79
147  IE         2018           69
148  IE         2019           77
149  IE         2020           76
